# Companion Notebook: Visualization Dashboard

This notebook loads the saved artifacts from `TrainedModels/` and provides interactive visualizations:

1. Feature correlation matrix
2. 2×2 confusion matrix selector (including baselines, tuned thresholds)
3. Decision & Random Forest tree visualizations
4. XGBoost feature importance & threshold tuning
5. Model performance comparison with selectable metrics and baseline inclusion via checkboxes

# 1. Imports and Configuration

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import xgboost as xgb
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from ipywidgets import Checkbox, Dropdown, interactive_output, VBox, HBox, interact

sns.set(style='whitegrid')
%matplotlib inline

# Load saved data
SAVE_DIR = 'TrainedModels'
thresh_df = pd.read_csv(os.path.join(SAVE_DIR, 'xgb_threshold_tuning.csv'))
perf_df   = pd.read_csv(os.path.join(SAVE_DIR, 'model_performance_summary.csv'))


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'xgboost'

## 2. Data Load & Correlation Matrix
Histogram with actual temperature difference denormalized to range °C (0–12.10 range) seen in original file, and correlation matrix

In [ ]:
# Load normalized data
dtype_dict = {
    'norm_power':'float32',
    'norm_temp_diff':'float32',
    'norm_tool_wear_adjusted':'float32',
    'Bool_MF':'bool'
}
data = pd.read_csv('full_normalized.csv', dtype=dtype_dict)
data['Bool_MF_int'] = data['Bool_MF'].astype('int8')

data['temp_diff_deg'] = data['norm_temp_diff'] * 12.10 # value manually calculated from source file.
plt.figure(figsize=(6,5))
sns.histplot(data['temp_diff_deg'], kde=True)
plt.title('Distribution of True Temp Difference (°C)')
plt.xlabel('Temp Difference (°C)')
plt.tight_layout()
plt.show()

# %%
# Normalized correlation
corr = data[['norm_power','norm_temp_diff','norm_tool_wear_adjusted','Bool_MF_int']].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation: Features + Failure')
plt.tight_layout()
plt.show()

# Prepare features and test split
after = data.drop(columns=['temp_diff_deg','Bool_MF_int'])
X = data[['norm_power','norm_temp_diff','norm_tool_wear_adjusted']]
y = data['Bool_MF_int']
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
del data


## 3. Interactive 2×2 Confusion Matrices

In [ ]:
def load_models():
    m = {}
    # Decision Tree
    m['DT Baseline'] = (joblib.load(os.path.join(SAVE_DIR,'DecisionTree_baseline.pkl')), None)
    m['DT Tuned']    = (joblib.load(os.path.join(SAVE_DIR,'DecisionTree_best.pkl')), None)
    # Random Forest
    m['RF Baseline'] = (joblib.load(os.path.join(SAVE_DIR,'RandomForest_baseline.pkl')), None)
    m['RF Tuned']    = (joblib.load(os.path.join(SAVE_DIR,'RandomForest_best.pkl')), None)
    # XGBoost
    base = xgb.Booster(); base.load_model(os.path.join(SAVE_DIR,'XGBoost_baseline.json'))
    tuned = xgb.Booster(); tuned.load_model(os.path.join(SAVE_DIR,'XGBoost_best.json'))
    m['XGB Baseline']     = (None, (base, 0.5))
    for th in thresh_df['threshold']:
        m[f"XGB Tuned ({th:.2f})"] = (None, (tuned, th))
    return m

models = load_models()
labels = list(models.keys())

@interact(
    tl=Dropdown(options=labels, description='Top-left'),
    tr=Dropdown(options=labels, description='Top-right'),
    bl=Dropdown(options=labels, description='Bottom-left'),
    br=Dropdown(options=labels, description='Bottom-right')
)
def plot_confusion_grid(tl, tr, bl, br):
    fig, axes = plt.subplots(2,2, figsize=(12,10))
    for ax, name in zip(axes.flatten(), [tl, tr, bl, br]):
        clf, info = models[name]
        if clf:
            preds = clf.predict(X_test)
        else:
            bst, thr = info
            proba = bst.predict(xgb.DMatrix(X_test))
            preds = (proba > thr).astype(int)
        cm = confusion_matrix(y_test, preds)
        sns.heatmap(cm, annot=True, fmt='d', ax=ax, cbar=False)
        ax.set_title(name)
        ax.set_xlabel('Predicted')
        ax.set_ylabel('Actual')
    plt.tight_layout()
    plt.show()



## 4. Decision & Random Forest Tree Visualizations
This section lets you inspect the structure of the first few levels of a Decision Tree or Random Forest (single tree).


In [ ]:
@interact(model=Dropdown(options=['DT Baseline','DT Tuned','RF Baseline','RF Tuned'],
                         description='Model'))
def plot_sklearn_trees(model):
    clf, _ = models[model]
    if model.startswith('RF'):
        clf = clf.estimators_[0]
    plt.figure(figsize=(12,6))
    plot_tree(clf,
              feature_names=['norm_power','norm_temp_diff','norm_tool_wear_adjusted'],
              class_names=['0','1'], filled=True, max_depth=3)
    plt.title(f"{model} (first tree, max depth=3)")
    plt.tight_layout()
    plt.show()

## 5. XGBoost Feature Importance & Threshold Tuning

In [ ]:
plt.figure(figsize=(6,4))
bst = xgb.Booster(); bst.load_model(os.path.join(SAVE_DIR,'XGBoost_best.json'))
xgb.plot_importance(bst, importance_type='weight')
plt.title('XGB Feature Importance (F-score)')
plt.tight_layout()
plt.show()

if 'f1' not in thresh_df.columns:
    thresh_df['f1'] = 2 * (thresh_df['precision'] * thresh_df['recall']) / (thresh_df['precision'] + thresh_df['recall'])
plt.figure(figsize=(8,5))
for col in ['accuracy','precision','recall','f1']:
    plt.plot(thresh_df['threshold'], thresh_df[col], marker='o', label=col)
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('XGB Threshold Tuning')
plt.legend()
plt.tight_layout()
plt.show()

## 6. Model Performance Comparison

 Compute train/test metrics for core models, including baseline XGB & tuned thresholds

In [ ]:
# Build model dict
comp_models = {
    'DT Baseline': models['DT Baseline'][0],
    'DT Tuned':    models['DT Tuned'][0],
    'RF Baseline': models['RF Baseline'][0],
    'RF Tuned':    models['RF Tuned'][0],
    'XGB Baseline':('xgb', models['XGB Baseline'][1]),
    'XGB Tuned (0.50)':('xgb', models['XGB Tuned (0.50)'][1])
}
th = [0.10,0.15,0.20,0.25,0.30]
for t in th:
    comp_models[f"XGB Tuned ({t:.2f})"] = models[f"XGB Tuned ({t:.2f})"]

# Widgets
metric_dd = Dropdown(options=['Accuracy','Precision','Recall','F1'], description='Metric')
# Default: baselines + 0.10,0.20,0.30 thresholds
defaults = [name for name in comp_models if 'Baseline' in name or any(name.endswith(f"({d:.2f})") for d in [0.10,0.20,0.30])]
model_checkboxes = {name: Checkbox(value=(name in defaults), description=name)
                    for name in comp_models}
# Arrange in 3 columns
from math import ceil
names = list(comp_models)
cols = ceil(len(names)/3)
groups = [names[i*cols:(i+1)*cols] for i in range(3)]
col_widgets = [VBox([model_checkboxes[n] for n in grp]) for grp in groups]
ui = VBox([metric_dd, HBox(col_widgets)])

# Plot function
# %%
def plot_selected(metric, **checks):
    sel = [n for n, v in checks.items() if v]
    rows = []
    for name in sel:
        item = comp_models[name]
        for split,Xd,yd in [('Train',X,y),('Test',X_test,y_test)]:
            if isinstance(item, tuple):
                bst, thr = item[1]
                proba = bst.predict(xgb.DMatrix(Xd))
                preds = (proba>thr).astype(int)
            else:
                preds = item.predict(Xd)
            acc  = accuracy_score(yd,preds)
            prec = precision_score(yd,preds)
            rec  = recall_score(yd,preds)
            f1   = 2*(prec*rec)/(prec+rec) if (prec+rec)>0 else 0
            rows.append({'Model':name,'Split':split,
                         'Accuracy':acc,'Precision':prec,'Recall':rec,'F1':f1})
    df = pd.DataFrame(rows)
    pivot = df.pivot(index='Model', columns='Split', values=[metric])
    pivot.columns = [f"{metric} {s}" for (_,s) in pivot.columns]
    pivot['Diff'] = pivot[f"{metric} Train"]-pivot[f"{metric} Test"]
    plot_df = pivot.reset_index().melt(id_vars='Model',
        value_vars=[f"{metric} Train",f"{metric} Test"], var_name='Split', value_name=metric)
    colors = sns.color_palette('muted')[:2]
    ax = sns.barplot(x=metric,y='Model',hue='Split',data=plot_df,
                     palette=colors,orient='h')
    plt.title(f"{metric}: Train vs Test (Diff shown)")
    xmin,xmax = plot_df[metric].min()-0.01, plot_df[metric].max()+0.01
    for idx,row in pivot.reset_index().iterrows():
        ax.text(xmin+0.02*(xmax-xmin), idx, f"{row['Diff']:.4f}",
                va='center', ha='left', bbox=dict(boxstyle='round,pad=0.2',facecolor='yellow',alpha=0.6))
    ax.get_legend().remove()
    plt.xlim(xmin,xmax)
    plt.tight_layout()
    plt.show()

# display main comparison
# %%
out = interactive_output(plot_selected, {'metric':metric_dd, **model_checkboxes})
display(ui, out)


### 6b. Metric Differences Only
Select models above and choose ordering to view only the train-test differences for all metrics.

In [ ]:
# Order by metric for diffs
order_dd = Dropdown(options=['Accuracy','Precision','Recall','F1'], description='Metric (Diff)')

# %%
def plot_diff_all(order_by, **checks):
    # Gather train-test diff for all metrics across selected models
    sel = [n for n,v in checks.items() if v]
    rows = []
    for name in sel:
        item = comp_models[name]
        # compute metrics for train/test
        mvals = {}
        for split,Xd,yd in [('Train',X,y),('Test',X_test,y_test)]:
            if isinstance(item, tuple):
                bst,thr = item[1]
                proba = bst.predict(xgb.DMatrix(Xd))
                preds = (proba>thr).astype(int)
            else:
                preds = item.predict(Xd)
            mvals[split] = {
                'Accuracy': accuracy_score(yd,preds),
                'Precision': precision_score(yd,preds),
                'Recall': recall_score(yd,preds),
                'F1':  2*(precision_score(yd,preds)*recall_score(yd,preds))/(precision_score(yd,preds)+recall_score(yd,preds)) if (precision_score(yd,preds)+recall_score(yd,preds))>0 else 0
            }
        # record diffs for each metric
        for metric in ['Accuracy','Precision','Recall','F1']:
            diff = mvals['Train'][metric] - mvals['Test'][metric]
            rows.append({'Model':name,'Metric':metric,'Diff':diff})
    df_diff = pd.DataFrame(rows)
    # determine model order based on selected metric
    order_df = df_diff[df_diff['Metric']==order_by].sort_values('Diff')
    order_list = order_df['Model'].tolist()
    # plot all metrics per model, sorted
    plt.figure(figsize=(8, max(4, len(order_list)*0.5)))
    ax = sns.barplot(x='Diff', y='Model', hue='Metric', data=df_diff, orient='h', order=order_list)
    plt.title(f'Train vs Test Differences Ordered by {order_by}')
    plt.tight_layout()
    plt.show()

# display diff-only plot with ordering
tout = interactive_output(plot_diff_all, {'order_by':order_dd, **model_checkboxes})
display(VBox([HBox(col_widgets), order_dd]), tout)


## 7. SHAP Explanation for XGB Tuned (0.20)
In this section, we use SHAP to explain the feature contributions for the XGBoost model tuned at 0.20 threshold.

In [ ]:
import matplotlib.colorbar as mcolorbar  # for custom colorbar
import shap
# helper to recreate and resize SHAP colorbars
def recreate_shap_colorbar(fig, width_multiplier=0.5, main_ax_index=0, collection_index=-1):
    main_ax = fig.axes[main_ax_index]
    if not main_ax.collections:
        print(f"Warning: no collections on axis {main_ax_index}")
        return
    col = main_ax.collections[collection_index]
    cmap, norm = col.get_cmap(), col.norm
    # remove existing colorbar
    cbar_ax = fig.axes[-1]
    label = cbar_ax.get_ylabel()
    cbar_ax.remove()
    # compute new position
    pos = cbar_ax.get_position()
    new_width = pos.width * width_multiplier
    new_x0 = pos.x0 - (new_width - pos.width)
    rect = [new_x0, pos.y0, new_width, pos.height]
    # create new colorbar
    new_ax = fig.add_axes(rect)
    cb = mcolorbar.ColorbarBase(new_ax, cmap=cmap, norm=norm, orientation='vertical')
    cb.set_label(label)

# Extract tuned XGBoost booster and threshold
_, (xgb_tuned, thr_020) = models['XGB Tuned (0.20)']

# Create explainer and sample data with 500 rows for speed
explainer = shap.TreeExplainer(xgb_tuned)
X_sample = X_test.sample(n=500, random_state=42)
shap_values = explainer.shap_values(X_sample)

# -- Summary (beeswarm) plot --
shap.summary_plot(shap_values, X_sample, feature_names=X_sample.columns, show=False)
fig = plt.gcf()
fig.set_size_inches(10, 6)
# resize colorbar
destroy = recreate_shap_colorbar(fig)
plt.show()

# -- Dependence plots for each feature --
for feat in X_sample.columns:
    shap.dependence_plot(feat, shap_values, X_sample, show=False)
    fig = plt.gcf()
    fig.set_size_inches(8, 6)
    recreate_shap_colorbar(fig, main_ax_index=0, collection_index=0)
    plt.show()
